In [74]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import gc
import numpy as np
import pickle
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from scipy import sparse
import lightgbm as lgb
from scipy.sparse import hstack, csr_matrix
import os
import glob

df_all = pickle.load(open('./data/basic.pkl','rb'))
df_train = df_all[df_all['deal_probability'].notnull()]
df_test = df_all[df_all['deal_probability'].isnull()].reset_index(drop=True)
y = df_all[df_all['deal_probability'].notnull()].deal_probability

# tfidf features
ready_df_train = sparse.load_npz('./data/features/nlp/ready_df_train_200000_new.npz')
ready_df_test = sparse.load_npz('./data/features/nlp/ready_df_test_200000_new.npz')
tfvocab = pickle.load(open('./data/features/nlp/tfvocab_200000_new.pkl', 'rb'))

# text agg features
for fn in glob.glob('./data/features/text_agg/train/*'):
    tmp = pickle.load(open(fn,'rb')).reset_index(drop=True)
    df_train[os.path.basename(fn)] = tmp
    del tmp
    gc.collect()
    #print (os.path.basename(fn))
    
for fn in glob.glob('./data/features/text_agg/test/*'):
    tmp = pickle.load(open(fn,'rb')).reset_index(drop=True)
    df_test[os.path.basename(fn)] = tmp
    del tmp
    gc.collect()
    #print (os.path.basename(fn))  

# numberical agg features
for fn in glob.glob('./data/features/number_agg/train/*'):
    tmp = pickle.load(open(fn,'rb'))
    df_train[os.path.basename(fn)] = tmp
    del tmp
    gc.collect()
    #print (os.path.basename(fn))
    
for fn in glob.glob('./data/features/number_agg/test/*'):
    tmp = pickle.load(open(fn,'rb'))
    df_test[os.path.basename(fn)] = tmp
    del tmp
    gc.collect()
    #print (os.path.basename(fn))        

/home/debutkele/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/debutkele/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
# price diff features
df_train['image_top_1_diff_price'] = df_train['price'] - df_train['image_top_1_median_price']
df_train['parent_category_name_diff_price'] = df_train['price'] - df_train['parent_category_name_mean_price']
df_train['category_name_diff_price'] = df_train['price'] - df_train['category_name_mean_price']
df_train['param_1_diff_price'] = df_train['price'] - df_train['param_1_mean_price']
df_train['param_2_diff_price'] = df_train['price'] - df_train['param_2_mean_price']
df_train['item_seq_number_diff_price'] = df_train['price'] - df_train['item_seq_number_mean_price']
df_train['user_id_diff_price'] = df_train['price'] - df_train['user_id_mean_price']
df_train['region_diff_price'] = df_train['price'] - df_train['region_mean_price']
df_train['city_diff_price'] = df_train['price'] - df_train['city_mean_price']

/home/debutkele/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/debutkele/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/debutkele/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [76]:
df_test['image_top_1_diff_price'] = df_test['price'] - df_test['image_top_1_median_price']
df_test['parent_category_name_diff_price'] = df_test['price'] - df_test['parent_category_name_mean_price']
df_test['category_name_diff_price'] = df_test['price'] - df_test['category_name_mean_price']
df_test['param_1_diff_price'] = df_test['price'] - df_test['param_1_mean_price']
df_test['param_2_diff_price'] = df_test['price'] - df_test['param_2_mean_price']
df_test['item_seq_number_diff_price'] = df_test['price'] - df_test['item_seq_number_mean_price']
df_test['user_id_diff_price'] = df_test['price'] - df_test['user_id_mean_price']
df_test['region_diff_price'] = df_test['price'] - df_test['region_mean_price']
df_test['city_diff_price'] = df_test['price'] - df_test['city_mean_price']

In [77]:
categorical = ['param_123']
for feature in categorical:
    print(f'Transforming {feature}...')
    encoder = LabelEncoder()
    encoder.fit(df_train[feature].append(df_test[feature]).astype(str))

Transforming param_123...


In [78]:
list(df_train.columns)

['activation_date',
 'category_name',
 'city',
 'deal_probability',
 'description',
 'image',
 'image_top_1',
 'item_id',
 'item_seq_number',
 'param_1',
 'param_2',
 'param_3',
 'parent_category_name',
 'price',
 'region',
 'title',
 'user_id',
 'user_type',
 'wday',
 'param_123',
 'text',
 'text_num_space',
 'param_123_num_words',
 'param_123_num_rus_low',
 'text_num_rus_cap',
 'title_num_rus_cap',
 'text_num_words_upper',
 'text_num_eng_cap',
 'text_num_rus_low',
 'param_123_num_rus_cap',
 'text_num_words',
 'title_num_low',
 'title_num_space',
 'title_num_unique_words',
 'title_num_pun',
 'text_ratio_unique_words',
 'title_num_eng_low',
 'param_123_num_eng_cap',
 'title_num_chars',
 'text_num_row',
 'param_123_num_unique_words',
 'text_num_pun',
 'text_num_words_lower',
 'param_123_num_low',
 'text_num_words_title',
 'title_ratio_unique_words',
 'param_123_num_dig',
 'param_123_num_eng_low',
 'title_num_cap',
 'text_num_dig',
 'param_123_num_cap',
 'text_num_chars',
 'title_num_dig

In [79]:
df_train.param_123

0                               Постельные принадлежности  
1                                                  Другое  
2                             Видео, DVD и Blu-ray плееры  
3                                    Автомобильные кресла  
4                                С пробегом ВАЗ (LADA) 2110
5                                    Автомобильные кресла  
6                                      Сантехника и сауна  
7                                  Женская одежда Джинсы 26
8                   Женская одежда Платья и юбки > 50 (XXL)
9                                      Для девочек Обувь 25
10                                Сдам На длительный срок 1
11                        Женская одежда Джинсы Без размера
12                                       Товары для школы  
13                                               Мониторы  
14                                       Немецкая овчарка  
15                   Женская одежда Платья и юбки 42–44 (S)
16                               С пробе

In [80]:
# create train test 
df_train = df_train.drop(['item_id','user_id','title','text','param_123','description','activation_date','image',
                'deal_probability'],axis=1)   
df_train['item_seq_number'] = np.log1p(df_train['item_seq_number'])
df_train.fillna(-1,inplace=True)

df_test = df_test.drop(['item_id','user_id','title','text','param_123','description','activation_date','image',
                'deal_probability'],axis=1) 
df_test['item_seq_number'] = np.log1p(df_test['item_seq_number'])
df_test.fillna(-1,inplace=True)

In [82]:
df_train = df_train.astype(np.float)
df_test = df_test.astype(np.float)

In [83]:
df_train.head()

,category_name,city,image_top_1,item_seq_number,param_1,param_2,param_3,parent_category_name,price,region,...,image_top_1_mean_price,image_top_1_diff_price,parent_category_name_diff_price,category_name_diff_price,param_1_diff_price,param_2_diff_price,item_seq_number_diff_price,user_id_diff_price,region_diff_price,city_diff_price
0,42.0,462.0,1008.0,1.098612,249.0,112.0,1217.0,4.0,5.993961,19.0,...,7.043741,-1.010013,-0.634113,-1.108448,-0.480276,-1.000000,-2.469891,-1.150169,-1.396188,-1.259907
1,22.0,1314.0,692.0,2.995732,122.0,112.0,1217.0,2.0,8.006701,17.0,...,8.613273,-0.249303,0.946285,0.294202,1.107127,-1.000000,0.608828,0.000000,0.479846,0.444528
2,2.0,1290.0,3032.0,2.302585,84.0,112.0,1217.0,0.0,8.294300,16.0,...,8.547559,-0.223094,0.820881,0.898418,1.284408,-1.000000,0.569325,1.462368,0.647754,0.605744
3,42.0,950.0,796.0,5.659482,38.0,112.0,1217.0,4.0,7.696667,21.0,...,7.713729,0.095265,1.068593,0.594258,-0.068886,-1.000000,0.306197,2.193171,0.134515,0.289292
4,0.0,318.0,2264.0,1.386294,278.0,124.0,46.0,6.0,10.596660,4.0,...,11.895838,-1.446900,-1.587835,-1.783630,-1.753453,-0.920605,2.381012,0.000000,3.236804,3.289208


In [84]:
list(df_train.dtypes)

[dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('floa

In [85]:
list(df_train.columns)

['category_name',
 'city',
 'image_top_1',
 'item_seq_number',
 'param_1',
 'param_2',
 'param_3',
 'parent_category_name',
 'price',
 'region',
 'user_type',
 'wday',
 'text_num_space',
 'param_123_num_words',
 'param_123_num_rus_low',
 'text_num_rus_cap',
 'title_num_rus_cap',
 'text_num_words_upper',
 'text_num_eng_cap',
 'text_num_rus_low',
 'param_123_num_rus_cap',
 'text_num_words',
 'title_num_low',
 'title_num_space',
 'title_num_unique_words',
 'title_num_pun',
 'text_ratio_unique_words',
 'title_num_eng_low',
 'param_123_num_eng_cap',
 'title_num_chars',
 'text_num_row',
 'param_123_num_unique_words',
 'text_num_pun',
 'text_num_words_lower',
 'param_123_num_low',
 'text_num_words_title',
 'title_ratio_unique_words',
 'param_123_num_dig',
 'param_123_num_eng_low',
 'title_num_cap',
 'text_num_dig',
 'param_123_num_cap',
 'text_num_chars',
 'title_num_dig',
 'text_num_stopwords',
 'text_num_unique_words',
 'param_123_ratio_unique_words',
 'param_123_num_pun',
 'text_num_eng_lo

In [ ]:
list(df_test.columns)

In [86]:
X_tr = hstack([csr_matrix(df_train),ready_df_train]) # Sparse Matrix
X_test = hstack([csr_matrix(df_test),ready_df_test])

In [ ]:
tfvocab = df_train.columns.tolist() + tfvocab
for shape in [X_tr,X_test]:
    print("{} Rows and {} Cols".format(*shape.shape))
print("Feature Names Length: ",len(tfvocab))

# train lightgbm model
from sklearn.model_selection import StratifiedKFold

X = X_tr.tocsr()
#del X_tra
gc.collect()

test_pred = np.zeros(X_test.shape[0])
cat_features=['region','city','parent_category_name',
              'category_name','user_type','image_top_1','wday','param_1']

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.019,
    'num_leaves': 300,  
    #'max_depth': 15,  
    'max_bin': 256,  
    'subsample': 0.7,  
    'colsample_bytree': 0.4,  
    'reg_alpha': 0,  
    'reg_lambda': 0,  
    'verbose': 1
    }

MAX_ROUNDS = 15000
NFOLDS = 5
kfold = KFold(n_splits=NFOLDS, shuffle=True, random_state=224)


for i,(train_index,val_index) in enumerate(kfold.split(X,y)):
    print("Running fold {} / {}".format(i + 1, NFOLDS))
    print("Train Index:",train_index,",Val Index:",val_index)
    X_tra,X_val,y_tra,y_val = X[train_index, :], X[val_index, :], y[train_index], y[val_index]
    if i >=0:

        dtrain = lgb.Dataset(
            X_tra, label=y_tra, feature_name=tfvocab, categorical_feature=cat_features)
        dval = lgb.Dataset(
            X_val, label=y_val, reference=dtrain, feature_name=tfvocab, categorical_feature=cat_features)    
        bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS,
            valid_sets=[dval], early_stopping_rounds=200, verbose_eval=200)

        del dtrain,dval
        del X_tra,y_tra,y_val,X_val
        gc.collect()

        test_pred = bst.predict(X_test, num_iteration=bst.best_iteration or MAX_ROUNDS)
        test_pred.dump('../models/kfold_' + str(i) + '.pkl')
        del test_pred
        gc.collect()

1503424 Rows and 1611656 Cols
508438 Rows and 1611656 Cols
Feature Names Length:  1611656
Running fold 1 / 5
Train Index: [      1       2       3 ... 1503421 1503422 1503423] ,Val Index: [      0       5      10 ... 1503410 1503413 1503414]


/home/debutkele/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/debutkele/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[200]	valid_0's rmse: 0.220564
[400]	valid_0's rmse: 0.218387
[600]	valid_0's rmse: 0.217323
[800]	valid_0's rmse: 0.21683
[1000]	valid_0's rmse: 0.216553
[1200]	valid_0's rmse: 0.216377
[1400]	valid_0's rmse: 0.216267
[1600]	valid_0's rmse: 0.216182
[1800]	valid_0's rmse: 0.216108
[2000]	valid_0's rmse: 0.216066
[2200]	valid_0's rmse: 0.216033
[2400]	valid_0's rmse: 0.216006
[2600]	valid_0's rmse: 0.215998
[2800]	valid_0's rmse: 0.215982
[3000]	valid_0's rmse: 0.215978
Early stopping, best iteration is:
[2880]	valid_0's rmse: 0.215971


/home/debutkele/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


Running fold 2 / 5
Train Index: [      0       1       2 ... 1503421 1503422 1503423] ,Val Index: [      3       7      12 ... 1503403 1503417 1503419]
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's rmse: 0.220256
[400]	valid_0's rmse: 0.217969
[600]	valid_0's rmse: 0.216927
[800]	valid_0's rmse: 0.216408
[1000]	valid_0's rmse: 0.216135
[1200]	valid_0's rmse: 0.215958
[1400]	valid_0's rmse: 0.215833
[1600]	valid_0's rmse: 0.215738
[1800]	valid_0's rmse: 0.215662
[2000]	valid_0's rmse: 0.215618
[2200]	valid_0's rmse: 0.215582
[2400]	valid_0's rmse: 0.215559
[2600]	valid_0's rmse: 0.215549
[2800]	valid_0's rmse: 0.215537
[3000]	valid_0's rmse: 0.215531
Early stopping, best iteration is:
[2959]	valid_0's rmse: 0.215528
Running fold 3 / 5
Train Index: [      0       2       3 ... 1503421 1503422 1503423] ,Val Index: [      1       4       6 ... 1503405 1503415 1503416]
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's rmse: 0.220

In [90]:
pred_1 = pickle.load(open('../models/kfold_0.pkl','rb'))
pred_2 = pickle.load(open('../models/kfold_1.pkl','rb'))
pred_3 = pickle.load(open('../models/kfold_2.pkl','rb'))
pred_4 = pickle.load(open('../models/kfold_3.pkl','rb'))
pred_5 = pickle.load(open('../models/kfold_4.pkl','rb'))

In [101]:
import pandas as pd
df_test = pd.read_csv("../input/test.csv")  

In [102]:
lgpred = (pred_1+pred_2+pred_3+pred_4+pred_5)/5

In [103]:
df_test.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
0,6544e41a8817,dbe73ad6e4b5,Волгоградская область,Волгоград,Личные вещи,Детская одежда и обувь,Для мальчиков,Обувь,25,Отдам бесплатно,На ангарском,NaN,66,2017-04-18,Private,a8b57acb5ab304f9c331ac7a074219aed4d349d8aef386...,2020.0
1,65b9484d670f,2e11806abe57,Свердловская область,Нижняя Тура,Хобби и отдых,Велосипеды,Дорожные,NaN,NaN,Продам велосипед,"Продам велосипед KAMA F200,в нормальном состо...",3000.0,4,2017-04-16,Private,NaN,NaN
2,8bab230b2ecd,0b850bbebb10,Новосибирская область,Бердск,Бытовая электроника,Аудио и видео,Телевизоры и проекторы,NaN,NaN,BBK,Продам новый телевизор BBK 32 диагональ смарт...,15000.0,15,2017-04-17,Private,8c361112cb049745ef2d1b0ae73594fc5c107286b0c942...,2960.0
3,8e348601fefc,5f1d5c3ce0da,Саратовская область,Саратов,Для дома и дачи,Бытовая техника,Для кухни,Вытяжки,NaN,Вытяжка Jetair 60,"Продам новую вытяжку в упаковке,с документами....",4500.0,70,2017-04-17,Private,NaN,NaN
4,8bd2fe400b89,23e2d97bfc7f,Оренбургская область,Бузулук,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,Коляска зима-лето,Продам отличную коляску. б/у 1 год. все вопрос...,4900.0,15,2017-04-15,Private,bc3cf6deef10840fc302e38eb48fa7748aa1e28d534f8f...,1002.0


In [104]:
lgsub = pd.DataFrame(lgpred,columns=["deal_probability"],index=df_test['item_id'])

In [105]:
lgsub['deal_probability'] = np.clip(lgsub['deal_probability'], 0.0, 1.0) # Between 0 and 1
lgsub.to_csv("lgsub.csv",index=True,header=True)

In [106]:
lgsub.head()

,deal_probability
item_id,
6544e41a8817,0.373715
65b9484d670f,0.100122
8bab230b2ecd,0.222271
8e348601fefc,0.091241
8bd2fe400b89,0.195442


In [107]:
lgsub.describe()

,deal_probability
count,508438.000000
mean,0.153034
std,0.147829
min,0.000000
25%,0.045714
50%,0.102229
75%,0.216171
max,1.000000
